In [1]:
from IPython.display import display, HTML
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from pathlib import Path

from time import sleep

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, multilabel_confusion_matrix
from sklearn.utils import shuffle

from importlib import reload
import sentinel_utils
import keras_model_creator

from data_generator import DataGenerator

In [2]:
sentinel_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12']
soilgrids_band = ['bdod', 'cec', 'cfvo', 'clay', 'nitrogen', 'ocd',
                  'ocs', 'phh2o', 'sand', 'silt', 'soc']
all_bands = sentinel_bands + ['Elevation'] + soilgrids_band
selected_bands = all_bands
bands = [all_bands.index(b) for b in selected_bands]
bands

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [3]:
reload(sentinel_utils)

shards_dir = Path.home().joinpath('sentinel_data', 'shards')

sentinel_shards = []
# seasons = ['03', '06', '09', '12']
seasons = ['03', '06', '09']
year = 2017
for s in seasons:
    path_list = list(shards_dir.joinpath(f'features_{year}{s}').glob('feature_*.npy'))
    sentinel_shards.extend(path_list)
    
sample_shards = 40000
utils = sentinel_utils.SentinelUtils(
    all_bands, seasons,
    shuffle(sentinel_shards, random_state=42)[:sample_shards],
    min_occurrences=20000, overwrite_existing=False
)
shard_ids = utils.selected_classes.index

In [4]:
loss = 'binary_crossentropy'
batch_size = 64
base_filters = 32
tag = 'conv3d+lstm'

model_dir = Path('models',
    f'{loss}-{len(shard_ids)}-{utils.selected_classes.shape[1]}'
    f'-{len(bands)}-{"_".join(seasons)}-{batch_size}-{base_filters}-{tag}'
)
model_dir.mkdir(parents=True, exist_ok=True)
model_dir

PosixPath('models/binary_crossentropy-237212-7-22-03_06_09-64-32-conv3d+lstm')

In [ ]:
reload(keras_model_creator)
params = dict(
    utils=utils,
    shards_dir=shards_dir,
    shard_ids=shard_ids,
    model_dir=model_dir,
    seasons=seasons,
    year=year,
    bands=bands,
    loss=loss,
    batch_size=batch_size,
    base_filters=base_filters,
    dropout=0.2,
    epochs=50,
    overwrite=False
)

model, testing_generator = keras_model_creator.KerasModelCreator(**params).run()

Previous training:


epoch,accuracy,val_accuracy,auc,val_auc,loss,val_loss,macrof1score,val_macrof1score,microf1score,val_microf1score,prc,val_prc,precision,val_precision,recall,val_recall,weightedf1score,val_weightedf1score
1,0.4862632155418396,0.5072115659713745,0.8917271494865417,0.917839527130127,0.271365612745285,0.3069191575050354,0.5298225283622742,0.5588865876197815,0.6547567248344421,0.7058344483375549,0.7285774350166321,0.7954415082931519,0.709792971611023,0.7623394131660461,0.6076412796974182,0.657127857208252,0.6429632902145386,0.6859409809112549
2,0.515836238861084,0.5269430875778198,0.9094045162200928,0.9177504777908324,0.2548909485340118,0.3082968890666961,0.572554886341095,0.6193166375160217,0.6957257390022278,0.7178924679756165,0.76884526014328,0.7907121181488037,0.7302755117416382,0.7302266955375671,0.6642975211143494,0.7059680819511414,0.68430495262146,0.7150656580924988
3,0.5268837809562683,0.5527844429016113,0.9149936437606812,0.9279778599739076,0.2480010390281677,0.2884266972541809,0.5904320478439331,0.6323546171188354,0.709244966506958,0.7360882759094238,0.7828414440155029,0.819110095500946,0.7355107069015503,0.7541114091873169,0.6847906112670898,0.7189066410064697,0.6988316774368286,0.7299807071685791
4,0.5369542241096497,0.5669070482254028,0.918922245502472,0.9257065057754515,0.2431318610906601,0.2955650389194488,0.6072355508804321,0.6215299963951111,0.7182996273040771,0.7320214509963989,0.7925843000411987,0.8066929578781128,0.7382240295410156,0.745811939239502,0.6994225382804871,0.7187317609786987,0.7097914814949036,0.7233749628067017
5,0.5416329503059387,0.5484775900840759,0.9213798642158508,0.9314101338386536,0.2397204488515854,0.2821762859821319,0.6166605353355408,0.6179453730583191,0.7230712175369263,0.7399109601974487,0.7981355786323547,0.8275293707847595,0.7389326691627502,0.7589312791824341,0.7078765630722046,0.7218207120895386,0.7158989310264587,0.7268328070640564
6,0.5515360832214355,0.5731169581413269,0.927769899368286,0.9351248145103456,0.2311324030160904,0.2750522494316101,0.6370172500610352,0.6516386866569519,0.7355163097381592,0.750377357006073,0.8133073449134827,0.8335611820220947,0.7447433471679688,0.7624059915542603,0.7265151143074036,0.7387224435806274,0.730280339717865,0.7440003156661987
epoch,accuracy,val_accuracy,auc,val_auc,loss,val_loss,macrof1score,val_macrof1score,microf1score,val_microf1score,prc,val_prc,precision,val_precision,recall,val_recall,weightedf1score,val_weightedf1score


Building model...
Fitting...
Epoch 7/50


I0000 00:00:1721260201.991616     129 service.cc:145] XLA service 0x9b4d810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721260201.991679     129 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
I0000 00:00:1721260229.448089     129 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3550/3550 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step - accuracy: 0.5394 - auc: 0.9335 - loss: 0.2751 - macrof1score: 0.5850 - microf1score: 0.7358 - prc: 0.8273 - precision: 0.7825 - recall: 0.6944 - weightedf1score: 0.7120

I0000 00:00:1721261670.077961     442 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1593', 192 bytes spill stores, 192 bytes spill loads

I0000 00:00:1721261670.149321     448 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1593', 308 bytes spill stores, 308 bytes spill loads



3550/3550 ━━━━━━━━━━━━━━━━━━━━ 1530s 421ms/step - accuracy: 0.5394 - auc: 0.9335 - loss: 0.2751 - macrof1score: 0.5850 - microf1score: 0.7358 - prc: 0.8273 - precision: 0.7825 - recall: 0.6944 - weightedf1score: 0.7120 - val_accuracy: 0.5278 - val_auc: 0.9406 - val_loss: 0.2617 - val_macrof1score: 0.6095 - val_microf1score: 0.7531 - val_prc: 0.8471 - val_precision: 0.7920 - val_recall: 0.7179 - val_weightedf1score: 0.7311 - learning_rate: 5.0000e-04
Epoch 8/50
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 1570s 425ms/step - accuracy: 0.5411 - auc: 0.9355 - loss: 0.2705 - macrof1score: 0.5869 - microf1score: 0.7405 - prc: 0.8330 - precision: 0.7871 - recall: 0.6991 - weightedf1score: 0.7156 - val_accuracy: 0.5388 - val_auc: 0.9411 - val_loss: 0.2596 - val_macrof1score: 0.5903 - val_microf1score: 0.7501 - val_prc: 0.8483 - val_precision: 0.8002 - val_recall: 0.7059 - val_weightedf1score: 0.7228 - learning_rate: 5.0000e-04
Epoch 9/50
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 1510s 424ms/step - accuracy: 0.5417 - a

In [ ]:
for eval_year in [2017, 2020, 2021, 2022, 2023]:
    params['year'] = eval_year
    eval_generator = DataGenerator(shard_ids, shuffle=False, **params)
    preds_path = model_dir.joinpath(f"preds_{params['year']}.npy")
    if preds_path.is_file():
        y_pred = np.load(preds_path)
    else:
        y_pred = model.predict(x=eval_generator, verbose=1)
        np.save(preds_path, y_pred)

    y_true = utils.selected_classes.iloc[:y_pred.shape[0]].to_numpy()
    cm = confusion_matrix(y_true.flatten(), (y_pred > 0.5).flatten().astype(int))
    plot = ConfusionMatrixDisplay(confusion_matrix=cm).plot()
    display(plot)

In [ ]:
# import matplotlib.pyplot as plt

# class_names = utils.selected_classes.columns

# f, axes = plt.subplots(6, 5, figsize=(25, 30))
# axes = axes.ravel()
# for label in range(y_true.shape[1]):
#     cm = confusion_matrix(y_true[..., label].astype(int), (y_pred[..., label] > 0.5).astype(int))
#     disp = ConfusionMatrixDisplay(cm)
#     disp.plot(ax=axes[label], values_format='.4g')
#     disp.ax_.set_title(f'{class_names[label]}')
#     if label < 25:
#         disp.ax_.set_xlabel('')
#     if label % 5 != 0:
#         disp.ax_.set_ylabel('')
#     disp.im_.colorbar.remove()

# plt.subplots_adjust(wspace=0.2, hspace=0.001)
# f.colorbar(disp.im_, ax=axes)
# plt.show()


In [ ]:
# import tensorflow
# tensorflow.keras.utils.plot_model(model, show_shapes=True)
# model.summary()

In [ ]:
tensorflow.keras.utils.plot_model(
    model, to_file=model_dir.joinpath('model.png'),
    show_shapes=True, show_layer_activations=True,
)

In [ ]:
# import tensorflow.keras.backend as K
# K.eval(model.optimizer.learning_rate)